# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [28]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

from sqlalchemy import create_engine
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [60]:
# load data from database
engine = create_engine('sqlite:///messages.db')
df = pd.read_sql("SELECT * FROM Messages_transformed", engine)
X = df.message.values
y = df.drop(columns=["id","message","original","genre","related"])
df.head()


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
X.shape

(26216,)

In [62]:
y.shape

(26216, 35)

### 2. Write a tokenization function to process your text data

In [63]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [64]:
pipeline = Pipeline([('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', RandomForestClassifier())
                     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [66]:
X_train.shape

(19662,)

In [67]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [72]:
y_pred = pipeline.predict(X_test)
labels = np.unique(y_pred)
accuracy = (y_pred == y_test).mean()

print("Labels:", labels)
print("Accuracy:", accuracy)

#print(type(y_pred))
#print(type(y_test))
report = classification_report(y_test, y_pred, target_names = y.columns.values)


Labels: [ 0.  1.]
Accuracy: request                   0.884803
offer                     0.994507
aid_related               0.721086
medical_help              0.919286
medical_products          0.949802
search_and_rescue         0.972231
security                  0.982148
military                  0.967196
child_alone               1.000000
water                     0.945224
food                      0.921269
shelter                   0.918523
clothing                  0.985047
money                     0.976808
missing_people            0.988862
refugees                  0.967043
death                     0.959414
other_aid                 0.869850
infrastructure_related    0.933323
transport                 0.952243
buildings                 0.950259
electricity               0.979554
tools                     0.994660
hospitals                 0.987489
shops                     0.995880
aid_centers               0.988709
other_infrastructure      0.956515
weather_related           0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [73]:
print(report)

                        precision    recall  f1-score   support

               request       0.81      0.40      0.54      1097
                 offer       0.00      0.00      0.00        36
           aid_related       0.78      0.44      0.56      2677
          medical_help       0.50      0.02      0.03       529
      medical_products       0.62      0.02      0.05       332
     search_and_rescue       0.67      0.01      0.02       183
              security       0.00      0.00      0.00       117
              military       0.67      0.01      0.02       216
           child_alone       0.00      0.00      0.00         0
                 water       0.87      0.16      0.27       414
                  food       0.87      0.31      0.46       706
               shelter       0.88      0.09      0.16       578
              clothing       1.00      0.04      0.08       102
                 money       0.33      0.01      0.01       151
        missing_people       0.00      

### 6. Improve your model
Use grid search to find better parameters. 

In [28]:
sorted(pipeline.get_params().keys())

['clf',
 'clf__bootstrap',
 'clf__class_weight',
 'clf__criterion',
 'clf__max_depth',
 'clf__max_features',
 'clf__max_leaf_nodes',
 'clf__min_impurity_decrease',
 'clf__min_impurity_split',
 'clf__min_samples_leaf',
 'clf__min_samples_split',
 'clf__min_weight_fraction_leaf',
 'clf__n_estimators',
 'clf__n_jobs',
 'clf__oob_score',
 'clf__random_state',
 'clf__verbose',
 'clf__warm_start',
 'cvect',
 'cvect__analyzer',
 'cvect__binary',
 'cvect__decode_error',
 'cvect__dtype',
 'cvect__encoding',
 'cvect__input',
 'cvect__lowercase',
 'cvect__max_df',
 'cvect__max_features',
 'cvect__min_df',
 'cvect__ngram_range',
 'cvect__preprocessor',
 'cvect__stop_words',
 'cvect__strip_accents',
 'cvect__token_pattern',
 'cvect__tokenizer',
 'cvect__vocabulary',
 'memory',
 'steps',
 'tfidf',
 'tfidf__norm',
 'tfidf__smooth_idf',
 'tfidf__sublinear_tf',
 'tfidf__use_idf']

In [31]:
parameters = {'clf__max_depth': [10, 20, None],
              'clf__min_samples_leaf': [1, 2, 4],
              'clf__min_samples_split': [2, 5, 10],
              'clf__n_estimators': [10, 20, 40]}



#pipeline.fit(X_train, y_train)

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', verbose=1, n_jobs=-1)

cv.fit(X_train, y_train)

Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed: 91.1min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__max_depth': [10, 20, None], 'clf__min_samples_leaf': [1, 2, 4], 'clf__min_samples_split': [2, 5, 10], 'clf__n_estimators': [10, 20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [32]:
y_pred = cv.predict(X_test)
labels = np.unique(y_pred)
accuracy = (y_pred == y_test).mean()

print("Labels:", labels)
print("Accuracy:", accuracy)

report = classification_report(y_test, y_pred)

Labels: [ 0.  1.]
Accuracy: request-0                   0.895941
offer-0                     0.995270
aid_related-0               0.750076
medical_help-0              0.924016
medical_products-0          0.954684
search_and_rescue-0         0.975130
security-0                  0.980623
military-0                  0.968264
child_alone-0               1.000000
water-0                     0.945987
food-0                      0.905096
shelter-0                   0.921269
clothing-0                  0.984437
money-0                     0.976808
missing_people-0            0.988862
refugees-0                  0.967806
death-0                     0.959262
other_aid-0                 0.872597
infrastructure_related-0    0.939121
transport-0                 0.955142
buildings-0                 0.951785
electricity-0               0.981843
tools-0                     0.995117
hospitals-0                 0.989319
shops-0                     0.996338
aid_centers-0               0.988862
other_infr

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [33]:
cv.best_params_

{'clf__max_depth': None,
 'clf__min_samples_leaf': 1,
 'clf__min_samples_split': 5,
 'clf__n_estimators': 40}

### 9. Export your model as a pickle file

In [36]:
m = pickle.dumps('clf')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.